In [144]:
import os
import sys
import re
import psutil
import time
import pickle 
import numpy as np
import pandas as pd
from collections import OrderedDict
from itertools  import groupby

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from sklearn.utils import shuffle
import random
random.seed(123)

In [2]:
Logpath='C:\ML_data\Logs'
labelpath='C:\ML_data\Logs'
print(os.listdir(Logpath))
logfilename='BGL.log'
bglfile = os.path.join(Logpath, logfilename)
print(os.path.exists(bglfile))
with open(bglfile, 'r',  encoding='utf8') as f:
    bglraw = f.readlines()
print(f'total loglines: {len(bglraw)}')

['anomaly_label.csv', 'anomaly_labels.txt', 'BGL.log', 'BGL.tar.gz', 'bgl2', 'bgl2.gz', 'BGL_sample.txt', 'HDFS.log', 'HDFS.log.zip', 'HDFS_1.tar.gz', 'HDFS_anomaly_label.csv', 'my_observed_anomaly_Openstack.txt', 'NeuralLog.zip', 'OpenStack.tar.gz', 'openstack_abnormal.log', 'OpenStack_log_files.zip', 'Spark.tar.gz']
True
total loglines: 4747963


In [3]:
# check few lines randomly to understand the pattern
random_indexs = random.sample(range(len(bglraw)), k=100)
for i in  random_indexs:
    print(bglraw[i])

KERNSTOR 1118765464 2005.06.14 R02-M1-N7-C:J09-U01 2005-06-14-09.11.04.290239 R02-M1-N7-C:J09-U01 RAS KERNEL FATAL data storage interrupt

- 1121493805 2005.07.15 R36-M1-ND-C:J17-U11 2005-07-15-23.03.25.354001 R36-M1-ND-C:J17-U11 RAS KERNEL INFO generating core.60016

- 1118772880 2005.06.14 R27-M0-N9-C:J03-U01 2005-06-14-11.14.40.067448 R27-M0-N9-C:J03-U01 RAS KERNEL FATAL lr:00004ed0 cr:28244842 xer:20000002 ctr:00086000

- 1127243236 2005.09.20 NULL 2005-09-20-12.07.16.840600 NULL RAS MMCS ERROR idoproxydb hit ASSERT condition: ASSERT expression=0 Source file=idotransportmgr.cpp Source line=1043 Function=int IdoTransportMgr::SendPacket(IdoUdpMgr*, BglCtlPavTrace*)

- 1121442094 2005.07.15 R01-M0-N2-C:J06-U01 2005-07-15-08.41.34.635525 R01-M0-N2-C:J06-U01 RAS KERNEL INFO generating core.1702

- 1119481844 2005.06.22 R22-M1-N4-C:J14-U01 2005-06-22-16.10.44.449189 R22-M1-N4-C:J14-U01 RAS KERNEL INFO 3091200 double-hummer alignment exceptions

KERNDTLB 1118550016 2005.06.11 R30-M0-N9-C:

In [4]:
l = bglraw[0].split()
l[8]

'INFO'

In [5]:
# alert_list = re.findall(r's\\')

In [6]:
# Documentation for this log suggests the label is amenable to the alert/severtiy information
# alert/severtiy  appears in the following two patterns:
# 1. after 'RAS KERNEL '  
# 2. it is also the 9th word from the left if we split the sentence by white space 

In [7]:
# get all the alerts in the list. total number of alerts should match with the 
#  total number of loglines
alerts =  [l.split()[8] for l in bglraw]
print(len(alerts))

4747963


In [8]:
unique_alerts = set(alerts)
print(f'unique_alerts: {unique_alerts}')

unique_alerts: {'Kill', 'microseconds', 'INFO', 'FATAL', '0x00544eb8,', 'ERROR', 'SEVERE', 'FAILURE', 'single', 'WARNING'}


In [9]:
negative_alerts = ['FATAL', 'SEVERE', 'WARNING', 'Kill', 'FAILURE', 'ERROR']

In [10]:
sequence_len = 32
print(len(bglraw) // sequence_len)
print((len(bglraw) + sequence_len - 1) // sequence_len)

148373
148374


In [11]:
mylist = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
window = 4
s1 = len(mylist)//window
s2 = (len(mylist)+window-1)//window
print(s1)
print(s2)
print(range(s1))
print(range(s2))
print([mylist[i*window:(i+1)*window] for i in range(s1)])
print([mylist[i*window:(i+1)*window] for i in range(s2)])

2
3
range(0, 2)
range(0, 3)
[[1, 2, 3, 4], [5, 6, 7, 8]]
[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10]]


In [12]:
sequences = [bglraw[i * sequence_len:(i + 1) * sequence_len] for i in range((len(bglraw)) // sequence_len )] 
print(len(sequences))

148373


In [13]:
# sequences[0]

In [14]:
txt = '- 1117838575 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.55.114809 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected\n'

In [15]:
if 'INFO' in txt: print('true')

true


In [16]:
# labelled_seq = [(s, 'INFO') for s in seq for seq in sequences if 'INFO' in s]
# labelled_seq

In [17]:
import time
stime = time.time()
labelled_sequences = []
for seq in sequences:    
    label = 'INFO'
    for s in seq:
        if s.split()[8] in negative_alerts:
            label = s.split()[8]
    labelled_sequences.append((seq, label))          
etime = time.time()
print(f'elapsed time: {etime - stime}')

elapsed time: 3.3191516399383545


In [18]:
df = pd.DataFrame(labelled_sequences, columns=['sequence', 'label'])
df.head()

,sequence,label
0,[- 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2...,INFO
1,[- 1117838575 2005.06.03 R02-M1-N0-C:J12-U11 2...,INFO
2,[- 1117838580 2005.06.03 R02-M1-N0-C:J12-U11 2...,INFO
3,[- 1117838584 2005.06.03 R02-M1-N0-C:J12-U11 2...,INFO
4,[- 1117838597 2005.06.03 R02-M1-N0-C:J12-U11 2...,INFO


In [19]:
posdf = df[df.label !='INFO']
posdf.head()

,sequence,label
155,[- 1117840351 2005.06.03 R16-M1-N2-C:J17-U01 2...,FATAL
156,[- 1117840660 2005.06.03 R26-M0-N4-I:J18-U01 2...,FATAL
157,[- 1117840730 2005.06.03 R22-M0-N4-I:J18-U01 2...,FATAL
460,[- 1117865192 2005.06.03 R12-M1-N3-C:J11-U11 2...,FATAL
461,[APPREAD 1117869873 2005.06.04 R36-M1-NC-I:J18...,FATAL


In [20]:
df.label.value_counts()

INFO       114115
FATAL       28439
ERROR        4049
WARNING       902
SEVERE        628
Kill          165
FAILURE        75
Name: label, dtype: int64

In [21]:
fataldf = df[df.label=='FATAL']
fataldf.sequence.values[0]

['- 1117840351 2005.06.03 R16-M1-N2-C:J17-U01 2005-06-03-16.12.31.760384 R16-M1-N2-C:J17-U01 RAS KERNEL INFO CE sym 2, at 0x1b85f480, mask 0x08\n',
 '- 1117840352 2005.06.03 R16-M1-N2-C:J17-U01 2005-06-03-16.12.32.025800 R16-M1-N2-C:J17-U01 RAS KERNEL INFO CE sym 2, at 0x1b85f680, mask 0x02\n',
 '- 1117840352 2005.06.03 R16-M1-N2-C:J17-U01 2005-06-03-16.12.32.239534 R16-M1-N2-C:J17-U01 RAS KERNEL INFO CE sym 2, at 0x1b85f8a0, mask 0x01\n',
 '- 1117840352 2005.06.03 R16-M1-N2-C:J17-U01 2005-06-03-16.12.32.427686 R16-M1-N2-C:J17-U01 RAS KERNEL INFO CE sym 2, at 0x1b85fa80, mask 0x04\n',
 '- 1117840352 2005.06.03 R16-M1-N2-C:J17-U01 2005-06-03-16.12.32.585331 R16-M1-N2-C:J17-U01 RAS KERNEL INFO CE sym 2, at 0x1b85fc80, mask 0x03\n',
 '- 1117840352 2005.06.03 R16-M1-N2-C:J17-U01 2005-06-03-16.12.32.854857 R16-M1-N2-C:J17-U01 RAS KERNEL INFO CE sym 2, at 0x1b85fe80, mask 0x02\n',
 '- 1117840353 2005.06.03 R16-M1-N2-C:J17-U01 2005-06-03-16.12.33.029069 R16-M1-N2-C:J17-U01 RAS KERNEL INFO CE 

In [22]:
# refresh the cell to randomly observer the nature of the fatal sequences
fataldf.sequence.values[random.choice(range(len(fataldf.sequence.values)))]

['KERNSTOR 1118767800 2005.06.14 R00-M1-N1-C:J07-U11 2005-06-14-09.50.00.377103 R00-M1-N1-C:J07-U11 RAS KERNEL FATAL data storage interrupt\n',
 'KERNSTOR 1118767800 2005.06.14 R00-M1-N1-C:J15-U01 2005-06-14-09.50.00.415246 R00-M1-N1-C:J15-U01 RAS KERNEL FATAL data storage interrupt\n',
 'KERNSTOR 1118767800 2005.06.14 R00-M1-N1-C:J17-U01 2005-06-14-09.50.00.442039 R00-M1-N1-C:J17-U01 RAS KERNEL FATAL data storage interrupt\n',
 'KERNSTOR 1118767800 2005.06.14 R00-M1-N1-C:J11-U01 2005-06-14-09.50.00.469806 R00-M1-N1-C:J11-U01 RAS KERNEL FATAL data storage interrupt\n',
 'KERNSTOR 1118767800 2005.06.14 R00-M1-N1-C:J07-U01 2005-06-14-09.50.00.497117 R00-M1-N1-C:J07-U01 RAS KERNEL FATAL data storage interrupt\n',
 'KERNSTOR 1118767800 2005.06.14 R00-M1-N1-C:J13-U01 2005-06-14-09.50.00.524050 R00-M1-N1-C:J13-U01 RAS KERNEL FATAL data storage interrupt\n',
 'KERNSTOR 1118767800 2005.06.14 R00-M1-N1-C:J09-U01 2005-06-14-09.50.00.551700 R00-M1-N1-C:J09-U01 RAS KERNEL FATAL data storage interr

In [23]:
manually_inspected = [
'- 1127163126 2005.09.19 R13-M1-NC-I:J18-U11 2005-09-19-13.52.06.454320 R13-M1-NC-I:J18-U11 RAS APP FATAL ciod: LOGIN chdir(/home/yates/SWL_tests/BGL64k_SWL_tests_develop/MPI-VAL/MPITs_v050902/rundir) failed: No such file or directory\n',

'APPSEV 1128267264 2005.10.02 R44-M1-NC-I:J18-U01 2005-10-02-08.34.24.605295 R44-M1-NC-I:J18-U01 RAS APP FATAL ciod: Error reading message prefix after LOAD_MESSAGE on CioStream socket to 172.16.96.116:58852: Link has been severed\n',
'- 1125667837 2005.09.02 R75-M1-N4-I:J18-U11 2005-09-02-06.30.37.373188 R75-M1-N4-I:J18-U11 RAS APP FATAL ciod: LOGIN chdir(/bglscratch/bwallen/SWL/SYS-CALLS/testcases/kernel/syscalls/socket) failed: No such file or directory\n',

'KERNMNTF 1125690086 2005.09.02 R74-M1-NC-I:J18-U01 2005-09-02-12.41.26.373742 R74-M1-NC-I:J18-U01 RAS KERNEL FATAL Lustre mount FAILED : bglio976 : point /p/gb1\n',

'KERNDTLB 1118536077 2005.06.11 R30-M0-N9-C:J16-U01 2005-06-11-17.27.57.437325 R30-M0-N9-C:J16-U01 RAS KERNEL FATAL data TLB error interrupt\n',
'- 1123112513 2005.08.03 R46-M1-NB-C:J10-U01 2005-08-03-16.41.53.337151 R46-M1-NB-C:J10-U01 RAS KERNEL FATAL write buffer commit threshold............2\n',

'- 1118770143 2005.06.14 R27-M1-N7-C:J08-U01 2005-06-14-10.29.03.025649 R27-M1-N7-C:J08-U01 RAS KERNEL FATAL auxiliary processor.........................0\n',]

In [24]:
# the message starts with '-', or 'APPSEV' or 'KERNMNTF' or 'KERNDTLB'
# next timestamp
# R13-M1-NC-I:J18-U11
# 2005-09-19-13.52.06.454320 - another timestamp
# R13-M1-NC-I:J18-U11
# RAS
# APP or KERNEL
# Severtiy 
# actaul mesage
# there is no relation between the severity/alert and the meta informations
# the prominent relationship is visibile between severtiy and the message type
# Hence we will remove the meta information except the severity and the message 

In [25]:
# {'UNK': 1, ' ': 2, 'e': 3, 'n': 4, 'r': 5, 'i': 6, 'o': 7, 't': 8, 'a': 9, 'l': 10, 'c': 11, '0': 12, 's': 13, 
#'f': 14, 'd': 15, 'g': 16, '.': 17, 'p': 18, 'm': 19, 'k': 20, '1': 21, 'u': 22, '4': 23, '2': 24, '6': 25, 'x': 26, '5': 27, '8': 28, 
#'3': 29, 'b': 30, 'h': 31, ':': 32, '7': 33, '9': 34, 'y': 35, '-': 36, 'v': 37, 'w': 38, '_': 39, '*': 40, 'j': 41, 'z': 42, '+': 43, 'q': 44, 
#'#': 45, '!': 46, '>': 47, '@': 48, '?': 49, '$': 50, "'": 51, '%': 52, '\x0f': 53, 'ý': 54, 'ï': 55, 'ã': 56, '`': 57, 'ð': 58, '²': 59, 'ù': 60, '·': 61, 
#'\x00': 62, '&': 63, '|': 64, '\x10': 65, '}': 66, 'ç': 67, '\x98': 68}
# print(tk.word_index)

In [26]:
# {'UNK': 1, 'e': 2, 'n': 3, 'r': 4, 'i': 5, 'o': 6, 't': 7, 'a': 8, 'l': 9, 'c': 10, '0': 11, 's': 12, 'f': 13, 'd': 14, 'g': 15, 
#  'p': 16, 'm': 17, 'k': 18, '1': 19, 'u': 20, '4': 21, '2': 22, '6': 23, 'x': 24, '5': 25, '8': 26, '3': 27, 'b': 28, 'h': 29, ':': 30, 
#  '7': 31, '9': 32, 'y': 33, 'v': 34, 'w': 35, '_': 36, '*': 37, 'j': 38, 'z': 39, '+': 40, 'q': 41, '#': 42, '!': 43, '>': 44, '@': 45, 
#  '?': 46, '$': 47, "'": 48, '%': 49, '\x0f': 50, 'ý': 51, 'ï': 52, 'ð': 53, '²': 54, 'ù': 55, '·': 56, '\x00': 57, '&': 58, '|': 59, '\x10': 60, '}': 61, 'ç': 62, '\x98': 63}

In [27]:
# {'UNK': 1, 'e': 2, 'n': 3, 'r': 4, 'i': 5, 'o': 6, 't': 7, 'a': 8, 'l': 9, 'c': 10, '0': 11, 's': 12, 'f': 13, 'd': 14, 'g': 15, 'p': 16,
#'m': 17, 'k': 18, '1': 19, 'u': 20, '4': 21, '2': 22, '6': 23, 'x': 24, '5': 25, '8': 26, '3': 27, 'b': 28, 'h': 29, '7': 30, '9': 31, 'y': 32, 'v': 33, 
#'w': 34, 'j': 35, 'z': 36, '+': 37, 'q': 38, '!': 39, '>': 40, '?': 41, '$': 42, "'": 43, 'ý': 44, 'ð': 45, '²': 46, 'ù': 47, '·': 48, '|': 49, '}': 50, 'ç': 51}

In [28]:
part_1 = '^-\s|^\w+\s'
part_2 = '\d{10}\s'
time_1 = '\d{4}\.\d{2}\.\d{2}\s'
part_4 = '\w\d{2}-\w\d-\w{2}-\w:\w\d{2}-\w\d{2}\s'
time_2 = '\d{4}-\d{2}-\d{2}-\d{2}\.\d{2}\.\d{2}\.\d{6}\s'
part_6 = 'RAS'
part_7 = '[\n]'
signs_n_punctuations = '\]|\[|\)|\(|\=|\,|\;|\/|\{|\}[$]|[@]|[#]|[%]|[_]|[*]|[&]|[ï]|[ã]|[`]|[ð]|[-]|[\x0f]|[\x00]|[\x10]|[\x98]|[ç]|[:]|\''
white_space = '\s'
multiple_dots = '\.+?'
pat =f'{part_1}|{part_2}|{time_1}|{part_4}|{time_2}|{part_6}\s|{part_7}|{signs_n_punctuations}|{white_space}|{multiple_dots}'

    
# pat =f'{part_1}|{part_2}|{time_1}|{part_4}|{time_2}|{part_6}\s|{part_7}'

for txt_line in manually_inspected:
    s = re.sub(pat, '', txt_line)
    print(txt_line)
    print(s)
    print()
    print()

- 1127163126 2005.09.19 R13-M1-NC-I:J18-U11 2005-09-19-13.52.06.454320 R13-M1-NC-I:J18-U11 RAS APP FATAL ciod: LOGIN chdir(/home/yates/SWL_tests/BGL64k_SWL_tests_develop/MPI-VAL/MPITs_v050902/rundir) failed: No such file or directory

APPFATALciodLOGINchdirhomeyatesSWLtestsBGL64kSWLtestsdevelopMPIVALMPITsv050902rundirfailedNosuchfileordirectory


APPSEV 1128267264 2005.10.02 R44-M1-NC-I:J18-U01 2005-10-02-08.34.24.605295 R44-M1-NC-I:J18-U01 RAS APP FATAL ciod: Error reading message prefix after LOAD_MESSAGE on CioStream socket to 172.16.96.116:58852: Link has been severed

APPFATALciodErrorreadingmessageprefixafterLOADMESSAGEonCioStreamsocketto172169611658852Linkhasbeensevered


- 1125667837 2005.09.02 R75-M1-N4-I:J18-U11 2005-09-02-06.30.37.373188 R75-M1-N4-I:J18-U11 RAS APP FATAL ciod: LOGIN chdir(/bglscratch/bwallen/SWL/SYS-CALLS/testcases/kernel/syscalls/socket) failed: No such file or directory

APPFATALciodLOGINchdirbglscratchbwallenSWLSYSCALLStestcaseskernelsyscallssocketfailedN

In [29]:
def clean_bgl(txt_line, clean_part_1=True, clean_part_2=True, clean_time_1=True, clean_part_4=True, clean_time_2=True, clean_part_6=True):
    part_1 = ''
    part_2 = ''
    time_1 = ''
    part_4 = ''
    time_2 = ''
    part_6 = ''
    if clean_part_1:
        part_1 = '^-\s|^\w+\s'
    if clean_part_2:
        part_2 = '\d{10}\s'
    if clean_time_1:
        time_1 = '\d{4}\.\d{2}\.\d{2}\s'
    if clean_part_4:
        part_4 = '\w\d{2}-\w\d-\w{2}-\w:\w\d{2}-\w\d{2}\s'
    if clean_time_2:
        time_2 = '\d{4}-\d{2}-\d{2}-\d{2}\.\d{2}\.\d{2}\.\d{6}\s'
    if clean_part_6:
        part_6 = 'RAS'
    part_7 = '[\n]'
    signs_n_punctuations = '\]|\[|\)|\(|\=|\,|\;|\/|\{|\}[$]|[@]|[#]|[%]|[_]|[*]|[&]|[ï]|[ã]|[`]|[ð]|[-]|[\x0f]|[\x00]|[\x10]|[\x98]|[ç]|[:]|\''
#     signs_n_punctuations = '\]|\[|\)|\(|\=|\,|\;|\/|\{|\}\$\@\#\%\_\*\&\|ï|ã|`|ð|\-'
#     signs_n_punctuations = '\]|\[|\)|\(|\=|\,|\;|\/|\{|\}[$]|[@]|[#]|[%]|[_]|[*]|[&]|[ï]|[ã]|`|ð|[-]|[\x0f]|[\x00]|[\x10]|[\x98]|[ç]|[:]'
    white_space = '\s'
    multiple_dots = '\.+?'
    pat =f'{part_1}|{part_2}|{time_1}|{part_4}|{time_2}|{part_6}\s|{part_7}|{signs_n_punctuations}|{white_space}|{multiple_dots}'
#     signs_n_punctuations = '\]|\[|\)|\(|\=|\,|\;|\/'
#     pat =f'{part_1}|{part_2}|{time_1}|{part_4}|{time_2}|{part_6}\s|{part_7}|{signs_n_punctuations}'
    s = re.sub(pat, '', txt_line)
    return s

In [30]:
# cleaned_sequences = [[clean_bgl(line)] for sequence, label in labelled_sequences for line in sequence ]

In [31]:
cleaned_labelled_sequences = []
for sequence, label in labelled_sequences:
    cleaned_seq = []
    for line in sequence:
        cleaned_line = clean_bgl(line)
        cleaned_line = cleaned_line.lower()
        cleaned_seq.append(cleaned_line)
    cleaned_labelled_sequences.append((cleaned_seq, label)) 
cleaned_labelled_sequences[0]

(['kernelinfoinstructioncacheparityerrorcorrected',
  'kernelinfoinstructioncacheparityerrorcorrected',
  'kernelinfoinstructioncacheparityerrorcorrected',
  'kernelinfoinstructioncacheparityerrorcorrected',
  'kernelinfoinstructioncacheparityerrorcorrected',
  'kernelinfoinstructioncacheparityerrorcorrected',
  'kernelinfoinstructioncacheparityerrorcorrected',
  'kernelinfoinstructioncacheparityerrorcorrected',
  'kernelinfoinstructioncacheparityerrorcorrected',
  'kernelinfoinstructioncacheparityerrorcorrected',
  'kernelinfoinstructioncacheparityerrorcorrected',
  'kernelinfoinstructioncacheparityerrorcorrected',
  'kernelinfoinstructioncacheparityerrorcorrected',
  'kernelinfoinstructioncacheparityerrorcorrected',
  'kernelinfoinstructioncacheparityerrorcorrected',
  'kernelinfoinstructioncacheparityerrorcorrected',
  'kernelinfoinstructioncacheparityerrorcorrected',
  'kernelinfoinstructioncacheparityerrorcorrected',
  'kernelinfoinstructioncacheparityerrorcorrected',
  'kernelinf

In [32]:
# whole_text_for_training = []
# for sequence, _ in cleaned_labelled_sequences:
#     for line in sequence:
#         whole_text_for_training.append(s)

In [33]:
whole_text_for_training = [line for sequence, _ in cleaned_labelled_sequences for line in sequence]
len(whole_text_for_training)

4747936

In [34]:
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(whole_text_for_training)
print(len(tk.word_index))
print(tk.word_index)

50
{'UNK': 1, 'e': 2, 'n': 3, 'r': 4, 'i': 5, 'o': 6, 't': 7, 'a': 8, 'l': 9, 'c': 10, '0': 11, 's': 12, 'f': 13, 'd': 14, 'g': 15, 'p': 16, 'm': 17, 'k': 18, '1': 19, 'u': 20, '4': 21, '2': 22, '6': 23, 'x': 24, '5': 25, '8': 26, '3': 27, 'b': 28, 'h': 29, '7': 30, '9': 31, 'y': 32, 'v': 33, 'w': 34, 'j': 35, 'z': 36, '+': 37, 'q': 38, '!': 39, '>': 40, '?': 41, '$': 42, 'ý': 43, 'ð': 44, '²': 45, 'ù': 46, '·': 47, '|': 48, '}': 49, 'ç': 50}


In [35]:
padded_char_len=64
padding_style='post'
truncating='pre'
line = cleaned_labelled_sequences[0][0][0]
txt_2_num = tk.texts_to_sequences([line])
padded_txt_to_num = pad_sequences(txt_2_num, maxlen=padded_char_len, 
                                              padding=padding_style, truncating=truncating)

In [36]:
print(line)
print(txt_2_num)
num_to_text = tk.sequences_to_texts(txt_2_num)
print(num_to_text)
print(padded_txt_to_num)
print(tk.sequences_to_texts(padded_txt_to_num))

kernelinfoinstructioncacheparityerrorcorrected
[[18, 2, 4, 3, 2, 9, 5, 3, 13, 6, 5, 3, 12, 7, 4, 20, 10, 7, 5, 6, 3, 10, 8, 10, 29, 2, 16, 8, 4, 5, 7, 32, 2, 4, 4, 6, 4, 10, 6, 4, 4, 2, 10, 7, 2, 14]]
['k e r n e l i n f o i n s t r u c t i o n c a c h e p a r i t y e r r o r c o r r e c t e d']
[[18  2  4  3  2  9  5  3 13  6  5  3 12  7  4 20 10  7  5  6  3 10  8 10
  29  2 16  8  4  5  7 32  2  4  4  6  4 10  6  4  4  2 10  7  2 14  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
['k e r n e l i n f o i n s t r u c t i o n c a c h e p a r i t y e r r o r c o r r e c t e d UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK']


In [37]:
# cleaned_labelled_sequences[0]

In [38]:
num_sequences = []
for seq, label in cleaned_labelled_sequences:
    num_seq = []
    for sline in seq:
        try:        
            num_line = tk.texts_to_sequences([sline])
            padded_num_line = pad_sequences(num_line, maxlen=padded_char_len, 
                                                  padding=padding_style, truncating=truncating)
            num_seq.append(padded_num_line[0])
        except Exception as e:
            print(e)
            print('line:', sline)   
            break
    num_sequences.append((num_seq, label)) 
num_sequences[0]


([array([18,  2,  4,  3,  2,  9,  5,  3, 13,  6,  5,  3, 12,  7,  4, 20, 10,
          7,  5,  6,  3, 10,  8, 10, 29,  2, 16,  8,  4,  5,  7, 32,  2,  4,
          4,  6,  4, 10,  6,  4,  4,  2, 10,  7,  2, 14,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
  array([18,  2,  4,  3,  2,  9,  5,  3, 13,  6,  5,  3, 12,  7,  4, 20, 10,
          7,  5,  6,  3, 10,  8, 10, 29,  2, 16,  8,  4,  5,  7, 32,  2,  4,
          4,  6,  4, 10,  6,  4,  4,  2, 10,  7,  2, 14,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
  array([18,  2,  4,  3,  2,  9,  5,  3, 13,  6,  5,  3, 12,  7,  4, 20, 10,
          7,  5,  6,  3, 10,  8, 10, 29,  2, 16,  8,  4,  5,  7, 32,  2,  4,
          4,  6,  4, 10,  6,  4,  4,  2, 10,  7,  2, 14,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
  array([18,  2,  4,  3,  2,  9,  5,  3, 13,  6,  5,  3, 12,  7,  4, 20, 10,
          7,  5,  6,  3, 10,  8, 10, 29, 

In [45]:
num_sequences[0][1]

'INFO'

In [46]:
print(tk.sequences_to_texts(num_sequences[0][0]))

['k e r n e l i n f o i n s t r u c t i o n c a c h e p a r i t y e r r o r c o r r e c t e d UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK', 'k e r n e l i n f o i n s t r u c t i o n c a c h e p a r i t y e r r o r c o r r e c t e d UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK', 'k e r n e l i n f o i n s t r u c t i o n c a c h e p a r i t y e r r o r c o r r e c t e d UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK', 'k e r n e l i n f o i n s t r u c t i o n c a c h e p a r i t y e r r o r c o r r e c t e d UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK', 'k e r n e l i n f o i n s t r u c t i o n c a c h e p a r i t y e r r o r c o r r e c t e d UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK', 'k e r n e l i n f o i n s t r u c t i o n c a c h e p a r i t y e r r o r c o r r e c t e d UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK

In [47]:
print(len(num_sequences))

148373


In [67]:
numdf = pd.DataFrame(num_sequences, columns=['seq', 'label'])
numdf.head()

,seq,label
0,"[[18, 2, 4, 3, 2, 9, 5, 3, 13, 6, 5, 3, 12, 7,...",INFO
1,"[[18, 2, 4, 3, 2, 9, 5, 3, 13, 6, 5, 3, 12, 7,...",INFO
2,"[[18, 2, 4, 3, 2, 9, 5, 3, 13, 6, 5, 3, 12, 7,...",INFO
3,"[[18, 2, 4, 3, 2, 9, 5, 3, 13, 6, 5, 3, 12, 7,...",INFO
4,"[[18, 2, 4, 3, 2, 9, 5, 3, 13, 6, 5, 3, 12, 7,...",INFO


In [68]:
numdf.label.value_counts()

INFO       114115
FATAL       28439
ERROR        4049
WARNING       902
SEVERE        628
Kill          165
FAILURE        75
Name: label, dtype: int64

In [155]:
# numdf["label"].replace({"INFO": "NORMALBGL", "FATAL": "FATALBGL", "ERROR": "ERRORBGL", 
#                      "WARNING": "WARNINGBGL", "SEVERE": "SEVEREBGL", "Kill": "KillBGL",
#                     "FAILURE": "FAILUREBGL"}, inplace=True)
numdf["label"].replace({"INFO": "0", "FATAL": "1", "ERROR": "2", 
                     "WARNING": "3", "SEVERE": "4", "Kill": "5",
                    "FAILURE": "6"}, inplace=True)
numdf["label"].replace({"NORMALBGL": "0", "FATALBGL": "1", "ERRORBGL": "2", 
                     "WARNINGBGL": "3", "SEVEREBGL": "4", "KillBGL": "5",
                    "FAILUREBGL": "6"}, inplace=True)

In [156]:
numdf.label.value_counts()

0    114115
1     28439
2      4049
3       902
4       628
5       165
6        75
Name: label, dtype: int64

In [157]:
numdf.head()

,seq,label
0,"[[18, 2, 4, 3, 2, 9, 5, 3, 13, 6, 5, 3, 12, 7,...",0
1,"[[18, 2, 4, 3, 2, 9, 5, 3, 13, 6, 5, 3, 12, 7,...",0
2,"[[18, 2, 4, 3, 2, 9, 5, 3, 13, 6, 5, 3, 12, 7,...",0
3,"[[18, 2, 4, 3, 2, 9, 5, 3, 13, 6, 5, 3, 12, 7,...",0
4,"[[18, 2, 4, 3, 2, 9, 5, 3, 13, 6, 5, 3, 12, 7,...",0


In [158]:
def train_test_split_class(bgldf, ablation = 4000, train_test_ratio = 0.8, label='NORMALBGL'):
    train_data=None
    test_data=None
    train_cnt = round(ablation * train_test_ratio)
    test_cnt = round(ablation * (1 - train_test_ratio))

    if train_cnt <= bgldf[bgldf.label==label].count()[0] :
      train_data = bgldf[bgldf.label==label][0:train_cnt]
    else:
        print(f'{label} class does not have {train_cnt} records, it has only {bgldf[bgldf.label==label].count()[0]} records')
    if test_cnt <= bgldf[bgldf.label==label].count()[0] :
      test_data = bgldf[bgldf.label==label][train_cnt:ablation]
    else:
        print(f'{label} class does not have {test_cnt} records, it has only {bgldf[bgldf.label==label].count()[0]} records')
    if train_data is not None:
        print(f'train_{label}:, {train_data.count()[0]}')
    if test_data is not None:
        print(f'test_{label}:, {test_data.count()[0]}')
    return train_data, test_data

train_normal, test_normal = train_test_split_class(numdf, ablation = 4000, train_test_ratio = 0.8, label='0')
train_fatal, test_fatal = train_test_split_class(numdf, ablation = 4000, train_test_ratio = 0.8, label='1')
train_error, test_error = train_test_split_class(numdf, ablation = 4000, train_test_ratio = 0.8, label='2')
train_warning, test_warning = train_test_split_class(numdf, ablation = 902, train_test_ratio = 0.8, label='3')
train_severe, test_severe = train_test_split_class(numdf, ablation = 628, train_test_ratio = 0.8, label='4')
train_kill, test_kill = train_test_split_class(numdf, ablation = 165, train_test_ratio = 0.8, label='5')
train_failure, test_failure = train_test_split_class(numdf, ablation = 75, train_test_ratio = 0.8, label='6')

train_0:, 3200
test_0:, 800
train_1:, 3200
test_1:, 800
train_2:, 3200
test_2:, 800
train_3:, 722
test_3:, 180
train_4:, 502
test_4:, 126
train_5:, 132
test_5:, 33
train_6:, 60
test_6:, 15


In [159]:
pd.concat([train_normal,train_error] )

,seq,label
0,"[[18, 2, 4, 3, 2, 9, 5, 3, 13, 6, 5, 3, 12, 7,...",0
1,"[[18, 2, 4, 3, 2, 9, 5, 3, 13, 6, 5, 3, 12, 7,...",0
2,"[[18, 2, 4, 3, 2, 9, 5, 3, 13, 6, 5, 3, 12, 7,...",0
3,"[[18, 2, 4, 3, 2, 9, 5, 3, 13, 6, 5, 3, 12, 7,...",0
4,"[[18, 2, 4, 3, 2, 9, 5, 3, 13, 6, 5, 3, 12, 7,...",0
...,...,...
107529,"[[2, 19, 11, 21, 27, 13, 20, 3, 10, 7, 5, 6, 3...",2
107530,"[[2, 19, 11, 21, 27, 13, 20, 3, 10, 7, 5, 6, 3...",2
107531,"[[2, 19, 11, 21, 27, 13, 20, 3, 10, 7, 5, 6, 3...",2
107532,"[[2, 19, 11, 21, 27, 13, 20, 3, 10, 7, 5, 6, 3...",2


In [162]:
# classes = ['NORMALBGL', 'FATALBGL', 'ERRORBGL', 'WARNINGBGL','SEVEREBGL', 'KillBGL', 'FAILUREBGL', ] 
classes = ['0', '1', '2', '3','4', '5', '6', ] 
def train_test_multi_class(bgldf, ablation=4000, train_test_ratio=0.8, classes=classes):
    train_data = []
    test_data = []
    for class_name in classes:
            trdata, tsdata = train_test_split_class(bgldf, ablation=ablation, 
                                                    train_test_ratio=train_test_ratio, 
                                                    label=class_name)
            if trdata is not None: train_data.append(trdata)
            if tsdata is not None: test_data.append(tsdata)
    
    train_df = pd.concat(train_data)
    test_df = pd.concat(test_data)
    return train_df, test_df    

In [226]:
train_df, test_df = train_test_multi_class(numdf, ablation=100)
train_df.label.value_counts()

train_0:, 80
test_0:, 20
train_1:, 80
test_1:, 20
train_2:, 80
test_2:, 20
train_3:, 80
test_3:, 20
train_4:, 80
test_4:, 20
train_5:, 80
test_5:, 20
6 class does not have 80 records, it has only 75 records
test_6:, 0


0    80
1    80
2    80
3    80
4    80
5    80
Name: label, dtype: int64

In [227]:
x_train = list(train_df.seq.values)
y_train = list(train_df.label.values)
y_train = to_categorical(y_train)
print(y_train[:2])
x_test = list(test_df.seq.values)
y_test = list(test_df.label.values)
y_test = to_categorical(y_test)
print(y_test[:2])

[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]


In [224]:
print(y_train[80:82])

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]]


In [225]:
vocab_size = len(tk.word_index)
vocab_size = len(tk.word_index)
print(f'vocab_size: {vocab_size}')
char_onehot = vocab_size

vocab_size: 50


In [189]:
# x_train = list(x_train)

In [192]:
import tensorflow as tf
B=32
# train_data = tf.data.Dataset.from_tensor_slices(x_train)
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(buffer_size=y_train.shape[0]).batch(B, drop_remainder=True)
print(train_data)
    
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_data = test_data.shuffle(buffer_size=y_test.shape[0]).batch(B, drop_remainder=True)
print(test_data)

<BatchDataset shapes: ((32, 32, 64), (32, 6)), types: (tf.int32, tf.float32)>
<BatchDataset shapes: ((32, 32, 64), (32, 6)), types: (tf.int32, tf.float32)>


In [204]:
train_data.element_spec[0].shape[2]

64

In [215]:
train_data.element_spec[1].shape[1]

6

In [206]:
embedding_weights = []
embedding_weights.append(np.zeros(vocab_size))
for char, i in tk.word_index.items(): # from 1 to 51
    onehot = np.zeros(vocab_size)
    onehot[i-1] = 1
    embedding_weights.append(onehot)
embedding_weights = np.array(embedding_weights)
    
input_size =[ train_data.element_spec[0].shape[1], train_data.element_spec[0].shape[2]]
embedding_size = vocab_size

embedding_layer = tf.keras.layers.Embedding(vocab_size+1,
                                                embedding_size,
                                                input_length=input_size,
                                                weights = [embedding_weights])

In [ ]:
# ValueError: Shape mismatch: The shape of labels (received (192,)) 
#         should equal the shape of logits except for the last dimension (received (32, 6)).

In [221]:
conv1d_set1 = 3
conv1d_set2 = 3
dense_neurons=2048
filters=64
kernel_size=3
maxpool_1=True
epochs=25

inputs = tf.keras.layers.Input(batch_shape=(B, train_data.element_spec[0].shape[1], train_data.element_spec[0].shape[2]), dtype='float64' )
x = tf.keras.layers.Embedding(input_dim=vocab_size+1,
                                output_dim=embedding_size,
                                input_length=train_data.element_spec[0].shape[2],
                                weights = [embedding_weights],
                                )(inputs)
for _ in range(conv1d_set1):
    x = tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
if maxpool_1:
    x = tf.keras.layers.MaxPooling2D(pool_size=(1, train_data.element_spec[0].shape[2]))(x)
    x = tf.reshape(x, (B, train_data.element_spec[0].shape[1], filters))        
    for _ in range(conv1d_set2):
        x = tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=(train_data.element_spec[0].shape[1]) )(x)
    x = tf.reshape(x, (B, filters))
if not maxpool_1:
    x = tf.keras.layers.Flatten()(x)       
x = tf.keras.layers.Dense(dense_neurons)(x)
outputs = tf.keras.layers.Dense(train_data.element_spec[1].shape[1], activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
print(model.summary())
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
          metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
hist = model.fit(train_data, validation_data=test_data, epochs=epochs) 

Model: "functional_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(32, 32, 64)]            0         
_________________________________________________________________
embedding_12 (Embedding)     (32, 32, 64, 50)          2550      
_________________________________________________________________
conv1d_54 (Conv1D)           (32, 32, 64, 64)          9664      
_________________________________________________________________
conv1d_55 (Conv1D)           (32, 32, 64, 64)          12352     
_________________________________________________________________
conv1d_56 (Conv1D)           (32, 32, 64, 64)          12352     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (32, 32, 1, 64)           0         
_________________________________________________________________
tf_op_layer_Reshape_16 (Tens [(32, 32, 64)]          

In [228]:
train_df, test_df = train_test_multi_class(numdf, ablation=1000)
train_df.label.value_counts()

train_0:, 800
test_0:, 200
train_1:, 800
test_1:, 200
train_2:, 800
test_2:, 200
train_3:, 800
test_3:, 102
4 class does not have 800 records, it has only 628 records
test_4:, 0
5 class does not have 800 records, it has only 165 records
5 class does not have 200 records, it has only 165 records
6 class does not have 800 records, it has only 75 records
6 class does not have 200 records, it has only 75 records


0    800
1    800
2    800
3    800
Name: label, dtype: int64

In [229]:
x_train = list(train_df.seq.values)
y_train = list(train_df.label.values)
y_train = to_categorical(y_train)
print(y_train[:2])
x_test = list(test_df.seq.values)
y_test = list(test_df.label.values)
y_test = to_categorical(y_test)
print(y_test[:2])

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]]


In [230]:
B=32
# train_data = tf.data.Dataset.from_tensor_slices(x_train)
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(buffer_size=y_train.shape[0]).batch(B, drop_remainder=True)
print(train_data)
    
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_data = test_data.shuffle(buffer_size=y_test.shape[0]).batch(B, drop_remainder=True)
print(test_data)

<BatchDataset shapes: ((32, 32, 64), (32, 4)), types: (tf.int32, tf.float32)>
<BatchDataset shapes: ((32, 32, 64), (32, 4)), types: (tf.int32, tf.float32)>


In [231]:
embedding_weights = []
embedding_weights.append(np.zeros(vocab_size))
for char, i in tk.word_index.items(): # from 1 to 51
    onehot = np.zeros(vocab_size)
    onehot[i-1] = 1
    embedding_weights.append(onehot)
embedding_weights = np.array(embedding_weights)
    
input_size =[ train_data.element_spec[0].shape[1], train_data.element_spec[0].shape[2]]
embedding_size = vocab_size

embedding_layer = tf.keras.layers.Embedding(vocab_size+1,
                                                embedding_size,
                                                input_length=input_size,
                                                weights = [embedding_weights])

In [232]:
conv1d_set1 = 3
conv1d_set2 = 3
dense_neurons=2048
filters=64
kernel_size=3
maxpool_1=True
epochs=25

inputs = tf.keras.layers.Input(batch_shape=(B, train_data.element_spec[0].shape[1], train_data.element_spec[0].shape[2]), dtype='float64' )
x = tf.keras.layers.Embedding(input_dim=vocab_size+1,
                                output_dim=embedding_size,
                                input_length=train_data.element_spec[0].shape[2],
                                weights = [embedding_weights],
                                )(inputs)
for _ in range(conv1d_set1):
    x = tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
if maxpool_1:
    x = tf.keras.layers.MaxPooling2D(pool_size=(1, train_data.element_spec[0].shape[2]))(x)
    x = tf.reshape(x, (B, train_data.element_spec[0].shape[1], filters))        
    for _ in range(conv1d_set2):
        x = tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding='same')(x)
    x = tf.keras.layers.MaxPooling1D(pool_size=(train_data.element_spec[0].shape[1]) )(x)
    x = tf.reshape(x, (B, filters))
if not maxpool_1:
    x = tf.keras.layers.Flatten()(x)       
x = tf.keras.layers.Dense(dense_neurons)(x)
outputs = tf.keras.layers.Dense(train_data.element_spec[1].shape[1], activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
print(model.summary())
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
          metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
hist = model.fit(train_data, validation_data=test_data, epochs=epochs) 

Model: "functional_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(32, 32, 64)]            0         
_________________________________________________________________
embedding_14 (Embedding)     (32, 32, 64, 50)          2550      
_________________________________________________________________
conv1d_60 (Conv1D)           (32, 32, 64, 64)          9664      
_________________________________________________________________
conv1d_61 (Conv1D)           (32, 32, 64, 64)          12352     
_________________________________________________________________
conv1d_62 (Conv1D)           (32, 32, 64, 64)          12352     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (32, 32, 1, 64)           0         
_________________________________________________________________
tf_op_layer_Reshape_18 (Tens [(32, 32, 64)]          